In [10]:
import random
import numpy as np
import pandas as pd
import utilization_waiting as ut
from time import clock

clock()
nurse = 8
doctor = 4
patient = random.randint(15, 25)  # random variable
patient_nurse = int(patient * 0.2)  # patients only to see nurse
patient_nurse_doctor = patient - patient_nurse  # patients to see nurse & doctor

# (mean, sd, min, max) random variables indicating the time(minutes) a patient needs to spent with a nurse/doctor
mean_time = random.randint(25, 35)
sd = 1
min_time = mean_time - 10
max_time = mean_time + 10

# nurse only
total_nurseonly_time = ut.sum_truncnorm(mean_time, sd, min_time, max_time, patient_nurse)
# nurse and doctor
total_nursemix_time = ut.sum_truncnorm(mean_time, sd, min_time, max_time, patient_nurse_doctor)
total_doctor_time = ut.sum_truncnorm(mean_time, sd, min_time, max_time, patient_nurse_doctor)

# utilization
utilization_nurse = ut.rate((total_nurseonly_time + total_nursemix_time), nurse)
utilization_doctor = ut.rate((total_doctor_time), doctor)
print("\nThe utilization of nurse is {}%".format(round(utilization_nurse * 100), 2))
print("The utilization of doctor is {}%".format(round(utilization_doctor * 100), 2))


list_patient = []
for i in range(patient):
    list_patient.append('patient_{}'.format(i+1))

total_time = 60
random_arrival = [random.random() for i in range(patient)]
sum_arrival = sum(random_arrival)
arrival_time = [total_time * i / sum_arrival for i in random_arrival]

# arrive_time = float(60/patient)

df_patients = pd.DataFrame({'Arrival_point': 0, 'Nurse_only': False,
                            'Waiting_nurse': 0, 'Meet_nurse_point': 0,
                            'Seeing_nurse': 0, 'After_nurse_point': 0,
                            'Waiting_doctor': 0, 'Meet_doctor_point': 0,
                            'Seeing_doctor': 0, 'After_doctor_point': 0}, index=list_patient)

for i in range(patient):
    df_patients.loc[list_patient[i], 'Arrival_point'] += arrival_time[i]

list_nurseonly_number = random.sample(list(range(patient)), patient_nurse)
for i in list_nurseonly_number:
    df_patients.loc['patient_{}'.format(i + 1), 'Nurse_only'] = True

arr_nurse = np.zeros(nurse)
arr_doctor = np.zeros(doctor)

df_patients['Seeing_nurse'] = ut.time_spend(mean_time, sd, min_time, max_time, patient)

for i in range(patient):
    if min(arr_nurse) <= df_patients.loc[list_patient[i], 'Arrival_point']:
        df_patients.loc[list_patient[i], 'Meet_nurse_point'] = df_patients.loc[list_patient[i], 'Arrival_point']
    else:
        df_patients.loc[list_patient[i], 'Meet_nurse_point'] = min(arr_nurse)
    df_patients.loc[list_patient[i], 'Waiting_nurse'] = df_patients.loc[list_patient[i], 'Meet_nurse_point'] - df_patients.loc[list_patient[i], 'Arrival_point']
    df_patients.loc[list_patient[i], 'After_nurse_point'] = df_patients.loc[list_patient[i], 'Meet_nurse_point'] + df_patients.loc[list_patient[i], 'Seeing_nurse']

    arr_nurse[np.argmin(arr_nurse)] += df_patients.loc[list_patient[i], 'Seeing_nurse']


list_mix_patient = [p for p in df_patients[df_patients['Nurse_only'] == False].index]
time_doctor = ut.time_spend(mean_time, sd, min_time, max_time, patient_nurse_doctor)
for i in range(patient_nurse_doctor):
    df_patients.loc[list_mix_patient[i], 'Seeing_doctor'] = time_doctor[i]

for i in range(patient_nurse_doctor):
    if min(arr_doctor) <= df_patients.loc[list_mix_patient[i], 'After_nurse_point']:
        df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] = df_patients.loc[list_mix_patient[i], 'After_nurse_point']
    else:
        df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] = min(arr_doctor)
    df_patients.loc[list_mix_patient[i], 'Waiting_doctor'] = df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] - df_patients.loc[list_mix_patient[i], 'After_nurse_point']
    arr_doctor[np.argmin(arr_doctor)] += df_patients.loc[list_mix_patient[i], 'Seeing_doctor']
    df_patients.loc[list_mix_patient[i], 'After_doctor_point'] = df_patients.loc[list_mix_patient[i], 'Meet_doctor_point'] + df_patients.loc[list_mix_patient[i], 'Seeing_doctor']


average_wait_nurse = round(df_patients['Waiting_nurse'].mean(), 2)
max_wait_nurse = round(df_patients['Waiting_nurse'].max(), 2)
average_wait_doctor = round(df_patients['Waiting_doctor'].sum() / patient_nurse_doctor, 2)
max_wait_doctor = round(df_patients['Waiting_doctor'].max(), 2)

print('\nThe average and maxium waiting time for nurse are {} minutes and {} minutes'.format(average_wait_nurse, max_wait_nurse))
print('The average and maxium waiting time for doctor are {} minutes and {} minutes'.format(average_wait_doctor, max_wait_doctor))

print('\nTotal running time is %-1.5ss' % clock())

df_patients


The utilization of nurse is 164.0%
The utilization of doctor is 269.0%

The average and maxium waiting time for nurse are 31.05 minutes and 67.67 minutes
The average and maxium waiting time for doctor are 5.89 minutes and 34.74 minutes

Total running time is 2.407s


,Arrival_point,Nurse_only,Waiting_nurse,Meet_nurse_point,Seeing_nurse,After_nurse_point,Waiting_doctor,Meet_doctor_point,Seeing_doctor,After_doctor_point
patient_1,5.496770,False,0.000000,5.496770,33.927024,39.423794,0.000000,39.423794,33.901638,73.325431
patient_2,3.302199,False,0.000000,3.302199,35.043568,38.345767,0.000000,38.345767,35.915704,74.261471
patient_3,5.465405,False,0.000000,5.465405,33.482989,38.948394,0.000000,38.948394,35.169488,74.117882
patient_4,3.528390,False,0.000000,3.528390,33.457575,36.985966,0.000000,36.985966,35.287499,72.273465
patient_5,2.548594,False,0.000000,2.548594,33.926526,36.475120,0.000000,36.475120,33.168918,69.644038
patient_6,0.416632,False,0.000000,0.416632,33.386497,33.803129,1.366359,35.169488,31.048093,66.217582
patient_7,3.324614,True,0.000000,3.324614,33.388279,36.712893,0.000000,0.000000,0.000000,0.000000
patient_8,2.596692,False,0.000000,2.596692,35.485501,38.082192,0.000000,38.082192,35.168055,73.250247
patient_9,5.081886,False,28.304611,33.386497,33.783971,67.170468,0.000000,67.170468,34.789638,101.960106
patient_10,2.719395,False,30.668884,33.388279,34.049902,67.438181,0.000000,67.438181,33.097451,100.535631
